In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

###  Store Part I results into DataFrame

##### Load the csv exported in Part I to a DataFrame

In [2]:
# Save path to data set in a variable
weather_data = pd.read_csv("../WeatherPy/Output_data_file")
weather_data.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lon,Max Temp,Wind Speed
0,Avarua,50,CK,1603933950,81,-21.21,-159.78,72.99,19.26
1,Butaritari,100,KI,1603933950,73,3.07,172.79,81.46,5.03
2,Cabo San Lucas,64,MX,1603933865,65,22.89,-109.91,77.00,3.87
3,Mataura,62,NZ,1603933950,72,-46.19,168.86,62.01,5.01
4,Airai,73,TL,1603933950,55,-8.93,125.41,74.66,3.22


In [3]:
# Import API key
from config import g_key

## Humidity Heatmap

##### Configure gmaps.
##### Use the Lat and Lng as locations and Humidity as the weight.
##### Add Heatmap layer to map.

In [4]:
#Configure gmaps
gmaps.configure(api_key = g_key)

#Use the Lat and Lng as locations and Humidity as the weight
locations = weather_data[["Lat","Lon"]]

humidity = weather_data["Humidity"]

In [5]:
# Plot Heatmap
fig = gmaps.figure()
max_intensity = np.max(humidity)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=10, point_radius=1)

#Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
max_intensity

100

 ## Create new DataFrame fitting weather criteria

#### Narrow down the cities to fit weather conditions.
#### Drop any rows will null values.

In [7]:
# Narrow down the cities to fit weather conditions

narrowed_city_df = weather_data.loc[(weather_data["Wind Speed"]<=10) & (weather_data["Cloudiness"]== 0) &  
                                   (weather_data["Max Temp"]>= 70) & (weather_data["Max Temp"]<= 80)].dropna()

narrowed_city_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lon,Max Temp,Wind Speed
11,Diu,0,IN,1603933952,50,20.71,70.98,75.27,5.97
27,Piraju,0,BR,1603933956,40,-23.19,-49.38,71.53,0.60
141,Gewanē,0,ET,1603933984,34,10.17,40.65,73.94,3.24
143,Laguna,0,US,1603933485,29,38.42,-121.42,73.00,2.91
156,Kodīnar,0,IN,1603933987,63,20.79,70.70,74.12,9.78
162,Frutal,0,BR,1603933988,75,-20.02,-48.94,76.21,6.44
195,Myitkyina,0,MM,1603933995,77,25.38,97.40,74.79,1.05
205,Alibag,0,IN,1603933997,73,18.64,72.88,77.52,4.25
208,Madera,0,US,1603933921,28,37.17,-119.83,72.00,1.86
211,Geraldton,0,AU,1603933998,44,-28.77,114.60,79.00,5.99


 # Hotel Map

##### Store into variable named hotel_df.
##### Add a "Hotel Name" column to the DataFrame.
##### Set parameters to search for hotels with 5000 meters.
##### Hit the Google Places API for each city's coordinates.
##### Store the first Hotel result into the DataFrame.
##### Plot markers on top of the heatmap.

In [8]:
#Store into variable named hotel_df
Hotel_df = narrowed_city_df.loc[:, ["City", "Country", "Lat", "Lon"]]

#Add a "Hotel Name" column to the DataFrame.
Hotel_df["Hotel Name"] = ""

Hotel_df

,City,Country,Lat,Lon,Hotel Name
11,Diu,IN,20.71,70.98,
27,Piraju,BR,-23.19,-49.38,
141,Gewanē,ET,10.17,40.65,
143,Laguna,US,38.42,-121.42,
156,Kodīnar,IN,20.79,70.70,
162,Frutal,BR,-20.02,-48.94,
195,Myitkyina,MM,25.38,97.40,
205,Alibag,IN,18.64,72.88,
208,Madera,US,37.17,-119.83,
211,Geraldton,AU,-28.77,114.60,


In [9]:
params = {
    
        "radius": 5000,
        "types": "lodging",
        "key" : g_key         
}
for index, row in Hotel_df.iterrows():
    
    Lat =row["Lat"]
    Lng = row["Lon"]
    City_Name = row["City"]
    
    params ["location"] = f"{Lat},{Lng}"
    print (f"Getting result for Index {index}: {City_Name}.")
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(url, params=params)
    response = response.json()
       
    try:
        print(f"Closest hotel in {City_Name} is {response['results'][0]['name']}.")
        Hotel_df.loc[index, "Hotel Name"] = response['results'][0]['name']
        
    except (KeyError, IndexError):
            print("No result,skipping....")
            

Getting result for Index 11: Diu.
Closest hotel in Diu is Hotel Relax Inn.
Getting result for Index 27: Piraju.
Closest hotel in Piraju is Hotel Beira Rio.
Getting result for Index 141: Gewanē.
No result,skipping....
Getting result for Index 143: Laguna.
Closest hotel in Laguna is Holiday Inn Express & Suites Elk Grove Central - Hwy 99.
Getting result for Index 156: Kodīnar.
Closest hotel in Kodīnar is Hotel Krishna Park, Kodinar.
Getting result for Index 162: Frutal.
Closest hotel in Frutal is Hotel Recanto Dos Ipês.
Getting result for Index 195: Myitkyina.
Closest hotel in Myitkyina is Shwe Phyu Hotel.
Getting result for Index 205: Alibag.
Closest hotel in Alibag is Radisson Blu Resort & Spa - Alibaug, India.
Getting result for Index 208: Madera.
No result,skipping....
Getting result for Index 211: Geraldton.
Closest hotel in Geraldton is Broadwater Mariner Resort.
Getting result for Index 241: Katsuura.
Closest hotel in Katsuura is Katsuura Hotel Mikazuki.
Getting result for Index 2

In [10]:
#Display the DataFrame
Hotel_df

,City,Country,Lat,Lon,Hotel Name
11,Diu,IN,20.71,70.98,Hotel Relax Inn
27,Piraju,BR,-23.19,-49.38,Hotel Beira Rio
141,Gewanē,ET,10.17,40.65,
143,Laguna,US,38.42,-121.42,Holiday Inn Express & Suites Elk Grove Central...
156,Kodīnar,IN,20.79,70.70,"Hotel Krishna Park, Kodinar"
162,Frutal,BR,-20.02,-48.94,Hotel Recanto Dos Ipês
195,Myitkyina,MM,25.38,97.40,Shwe Phyu Hotel
205,Alibag,IN,18.64,72.88,"Radisson Blu Resort & Spa - Alibaug, India"
208,Madera,US,37.17,-119.83,
211,Geraldton,AU,-28.77,114.60,Broadwater Mariner Resort


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in Hotel_df.iterrows()]
locations = Hotel_df[["Lat", "Lon"]]

In [12]:
 # Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

#Add the layer to the map
fig.add_layer(markers)


# Display Map
fig

Figure(layout=FigureLayout(height='420px'))